In [1]:
#import relevent libraries
import numpy as np
import pandas as pd
import matplotlib as plt 
print("Libraries imported successfully")

Libraries imported successfully


In [2]:
#load train and test datasets
train = pd.read_csv("datasets/train.csv")
test = pd.read_csv("datasets/test.csv")

train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
input=np.asfarray(train.iloc[:,1:])

output=[]
for i in range(42000):
    out =np.zeros(10)+0.01
    out[train['label'][i]]=0.99
    output.append(out)


    
input_nodes = 784
num_layer = 3 # number of layers
h1 = 200       # hidden layer 1
h2 = 150        # hidden layer 2
output_nodes = 10
    
print("Input:")
print(input)
print("Output:")
print(output)




Input:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Output:
[array([0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99, 0.01]), array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.99]), 

In [4]:
out=output.shape
print(out)

(10,)


In [111]:
print("The size of input is {}.".format(input_nodes))
print("The size of output is {}.".format(output_nodes))

The size of input is 784.
The size of output is 10.


In [112]:
def sigmoid(z):
    g=1/(1+np.exp(-z))
    
    return g

In [113]:
def parameters_initialization(input_nodes, h1, h2, output_nodes):
    W1=np.random.randn(h1, input_nodes)
    W2=np.random.randn(h2, h1)
    W3=np.random.randn(output_nodes, h2)
    b1=np.zeros((h1,1))
    b2=np.zeros((h2,1))
    b3=np.zeros((output_nodes,1))
    
    return W1,W2,W3,b1,b2,b3

In [114]:
def forward_propagation(input, W1, W2, W3, b1, b2, b3):
    Z1=np.sum(np.dot(W1,input),b1)
    A1=sigmoid(Z1)
    Z2=np.sum(np.dot(W2,A1),b2)
    A2=sigmoid(Z2)
    Z3=np.sum(np.dot(W3,A2),b3)
    A3=sigmoid(Z3)
    
    return A3
    

In [115]:
def cost_function(output,A3):
    m=output.shape()[0]
    cost=(-1/m)*np.sum(output*np.log(A3)+(1-output*np.log(1-A3)))
    
    return cost


In [3]:
def back_propagation(input,output,A3,A2,A1,W3,W2,W1):
    m=input.shape()[0]
    dZ3=A3-output
    dZ2=np.dot(W3.T,dZ3)*(1-np.power(A2,2))
    dZ1=np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    
    dW3=(1/m)*np.dot(dZ3,A2.T)
    dW2=(1/m)*np.dot(dZ2,A1.T)
    dW1=(1/m)*np.dot(dZ1,input.T)
    
    db3=(1/m)*np.sum(dZ3, axis=1, keepdims=True)
    db2=(1/m)*np.sum(dZ2, axis=1, keepdims=True)
    db1=(1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1,dW2,dW3,db1,db2,db3 
    
    

In [117]:
def update_parameters(learning_rate,dW1,dW2,dW3,db1,db2,db3):
    W1=W1-learning_rate*dW1
    W2=W2-learning_rate*dW2
    W3=W3-learning_rate*dW3
    
    b1=b1-learning_rate*db1
    b2=b2-learning_rate*db2
    b3=b3-learning_rate*db3
    
    return W1,W2,W3,b1,b2,b3
    